## Tarbimise ja tootmise andmete allalaadimine

- loeme CSV failist tarbimise ja tootmise andmed.
- salvestame andmed faili `data/raw_consumption.json` ja `data/raw_production.json`

In [ ]:
import pandas as pd
from pathlib import Path

In [15]:
project_root = Path("..").resolve()
data_dir = project_root / "data"
data_dir.mkdir(exist_ok=True)

Laeme andmes .csv failist.

In [29]:
csv_path = Path("..") /"data"/"electricity_production_and_consumption_10.12-11.12.csv"

df = pd.read_csv(csv_path, sep=";", encoding="latin1")
df.head()

,Ajatempel (UTC),Kuupäev (Eesti aeg),Tarbimine,Tootmine,Planeeritud tarbimine,Planeeritud tootmine
0,1765317600,10.12.2025 00:00,"923,18","440,98","823,4",401
1,1765318500,10.12.2025 00:15,"929,09","433,93","817,6","373,7"
2,1765319400,10.12.2025 00:30,"901,28","435,96","807,9",376
3,1765320300,10.12.2025 00:45,"910,34","427,05","823,9","377,2"
4,1765321200,10.12.2025 01:00,"860,79","439,33","760,3",349


Puhastame veerud.

In [30]:
numeric_cols = ["Tarbimine", "Tootmine", "Planeeritud tarbimine", "Planeeritud tootmine"]

for col in numeric_cols:
    df[col] = df[col].astype(str).str.replace(",", ".").astype(float)

df.head()

,Ajatempel (UTC),Kuupäev (Eesti aeg),Tarbimine,Tootmine,Planeeritud tarbimine,Planeeritud tootmine
0,1765317600,10.12.2025 00:00,923.18,440.98,823.4,401.0
1,1765318500,10.12.2025 00:15,929.09,433.93,817.6,373.7
2,1765319400,10.12.2025 00:30,901.28,435.96,807.9,376.0
3,1765320300,10.12.2025 00:45,910.34,427.05,823.9,377.2
4,1765321200,10.12.2025 01:00,860.79,439.33,760.3,349.0


Lisame Timestampi

In [31]:
df["timestamp"] = pd.to_datetime(df["Ajatempel (UTC)"], unit="s")
df = df.sort_values("timestamp").reset_index(drop=True)

df[["Ajatempel (UTC)", "timestamp"]].head()

,Ajatempel (UTC),timestamp
0,1765317600,2025-12-09 22:00:00
1,1765318500,2025-12-09 22:15:00
2,1765319400,2025-12-09 22:30:00
3,1765320300,2025-12-09 22:45:00
4,1765321200,2025-12-09 23:00:00


Koostame tarbimise JSON faili.

In [32]:
df_consumption = df[["timestamp", "Tarbimine"]].copy()
df_consumption.rename(columns={"Tarbimine": "consumption_mwh"}, inplace=True)

df_consumption.to_json(data_dir / "raw_consumption.json", orient="records", date_format="iso")

df_consumption.head()


,timestamp,consumption_mwh
0,2025-12-09 22:00:00,923.18
1,2025-12-09 22:15:00,929.09
2,2025-12-09 22:30:00,901.28
3,2025-12-09 22:45:00,910.34
4,2025-12-09 23:00:00,860.79


Koostame tootmise JSON faili.

In [33]:
df_production = df[["timestamp", "Tootmine"]].copy()
df_production.rename(columns={"Tootmine": "production_mwh"}, inplace=True)

df_production.to_json(data_dir / "raw_production.json", orient="records", date_format="iso")

df_production.head()

,timestamp,production_mwh
0,2025-12-09 22:00:00,440.98
1,2025-12-09 22:15:00,433.93
2,2025-12-09 22:30:00,435.96
3,2025-12-09 22:45:00,427.05
4,2025-12-09 23:00:00,439.33


Koostame planeeritud tarbimise JSON faili.

In [34]:
df_forecast = df[["timestamp", "Planeeritud tarbimine"]].copy()
df_forecast.rename(columns={"Planeeritud tarbimine": "forecast_consumption_mwh"}, inplace=True)

forecast_path = data_dir / "raw_forecast_consumption.json"
df_forecast.to_json(forecast_path, orient="records", date_format="iso")

forecast_path, df_forecast.head()

(WindowsPath('C:/Users/tonis/Desktop/Balance-Process-Simulator/data/raw_forecast_consumption.json'),
             timestamp  forecast_consumption_mwh
 0 2025-12-09 22:00:00                     823.4
 1 2025-12-09 22:15:00                     817.6
 2 2025-12-09 22:30:00                     807.9
 3 2025-12-09 22:45:00                     823.9
 4 2025-12-09 23:00:00                     760.3)

Koostame planeeritud tootmise JSON faili.

In [35]:
df_forecast = df[["timestamp", "Planeeritud tootmine"]].copy()
df_forecast.rename(columns={"Planeeritud tootmine": "forecast_production_mwh"}, inplace=True)

forecast_path = data_dir / "raw_forecast_production.json"
df_forecast.to_json(forecast_path, orient="records", date_format="iso")

forecast_path, df_forecast.head()

(WindowsPath('C:/Users/tonis/Desktop/Balance-Process-Simulator/data/raw_forecast_production.json'),
             timestamp  forecast_production_mwh
 0 2025-12-09 22:00:00                    401.0
 1 2025-12-09 22:15:00                    373.7
 2 2025-12-09 22:30:00                    376.0
 3 2025-12-09 22:45:00                    377.2
 4 2025-12-09 23:00:00                    349.0)

- Importisin tarbimise ja tootmise andmed CSV failist.
- Normaliseerisin numbriveerud.
- Teisendasin timestampi UNIX sekunditest kuupäevaks.
- Lõin JSON failid:
    - `raw_consumption.json`
    - `raw_production.json`
    - `raw_forecast_consumption.json`
    - `raw_forecast_production.json`